## Analyse Appliences on SmartMeters in Buildings 
### UKDALE2 Dataset

In [55]:
import nilmtk

start_time = '2023-08-01'
end_time = '2023-09-06'
sample_period = 10


ukdale = nilmtk.DataSet("ukdale2.h5")
ukdale.set_window(start=start_time, end=end_time)


arr = []
for j in range(len(ukdale.buildings)):
    arr2 = []
    for i in range(len(ukdale.buildings[j+1].elec.appliances)):
        arr2.append(ukdale.buildings[j+1].elec.appliances[i].type["type"])
    arr.append(arr2)

common_apps = set(arr[0]).intersection(arr[1]) # Übereinstimmungen aus Building 1 und 2
print(common_apps)

{'computer monitor', 'dish washer', 'broadband router', 'toaster', 'kettle', 'computer', 'laptop computer', 'external hard disk', 'microwave'}


In [11]:
buildings = [1,2,5]
# len(ukdale.buildings[1].elec.meters)
metersPerBuilding = [53,19,5,6,25]
single_apps = [[],[],[]]

for i in range(3):
    for j in ukdale.buildings[i+1].elec.meters:
        if len(j.appliances) == 1:
            single_apps[i].append(j.appliances[0].type['type'])
print(set(single_apps[0])) # single apps aus Building 1
print(set(single_apps[1])) # single apps aus Building 2
print(set(single_apps[2])) # single apps aus Building 5

{'radio', 'oven', 'solar thermal pumping station', 'desktop computer', 'mobile phone charger', 'baby monitor', 'clothes iron', 'light', 'breadmaker', 'vacuum cleaner', 'dish washer', 'microwave', 'tablet computer charger', 'computer monitor', 'television', 'coffee maker', 'soldering iron', 'fan', 'audio amplifier', 'audio system', 'hair dryer', 'HTPC', 'printer', 'charger', 'hair straighteners', 'active subwoofer', 'fridge freezer', 'boiler'}
{'computer monitor', 'rice cooker', 'computer', 'active speaker', 'broadband router', 'laptop computer', 'fridge', 'games console', 'modem', 'external hard disk', 'dish washer', 'cooker', 'toaster', 'running machine', 'microwave', 'kettle', 'washing machine'}
{'electric space heater', 'kettle', 'projector', 'laptop computer'}


In [12]:
# common_single_apps = set(single_apps[0]).intersection(single_apps[1],single_apps[2]) # keine Übereinstimmungen
common_single_apps = set(single_apps[0]).intersection(single_apps[1]) # Übereinstimmungen von single apps aus Building 1 und 2
common_single_apps

{'computer monitor', 'dish washer', 'microwave'}

In [13]:
apps = common_single_apps.intersection(common_apps) # Übereinstimmungen von common und single apps aus Building 1 und 2 
apps

{'dish washer'}

## Check TimeRanges for Appliances

In [ ]:
import nilmtk
import pandas as pd
ukdale = nilmtk.DataSet("ukdale2.h5")

# {'baby monitor', 'radio', 'toasted sandwich maker', 'laptop computer', 'solar thermal pumping station', 'light', 'external hard disk', 'dish washer', 'mobile phone charger', 'toaster', 'USB hub', 'hair dryer', 'kitchen aid', 'boiler', 'security alarm'}

apps = ['microwave', 'dish washer', 'kettle', 'broadband router','computer monitor','laptop computer','external hard disk','computer']
timerange = []
temp = []
b = []
a = []
t = []
c=[]


for i in apps:
    temp = [i]
    for j in range(5):
        try:
            df = next(ukdale.buildings[j+1].elec[i].load())
            temp.append((f'building {j+1}',df.index.min(),df.index.max()))
            timerange.append(temp)
            # start
            b.append(f'building {j+1}')
            a.append(i)
            t.append(df.index.min())
            c.append('start')
            # end
            b.append(f'building {j+1}')
            a.append(i)
            t.append(df.index.max())
            c.append('end')
        except:
            # print(f'{i} not in building {j+1}')
            pass


tdf = pd.DataFrame(data=[b,a,t,c]).transpose()
# tdf[(tdf[0] != 'building 3') & (tdf[0] != 'building 4') & (tdf[0] != 'building 5')].sort_values(by=2, ascending=True)
# tdf[tdf[0] == 'building 1'].sort_values(by=2, ascending=True)

In [65]:
# tdf[(tdf[0] != 'building 3') & (tdf[0] != 'building 4') & (tdf[0] != 'building 5')].sort_values(by=2, ascending=True)
tdf[tdf[0] == 'building 1'].sort_values(by=2, ascending=True)

,0,1,2,3
8,building 1,dish washer,2012-11-09 22:28:18+00:00,start
14,building 1,kettle,2012-11-09 22:28:18+00:00,start
34,building 1,laptop computer,2012-11-16 11:29:01+00:00,start
24,building 1,broadband router,2012-12-14 22:21:32+00:00,start
0,building 1,microwave,2012-12-14 22:21:33+00:00,start
28,building 1,computer monitor,2012-12-14 22:21:33+00:00,start
40,building 1,external hard disk,2013-03-14 20:20:02+00:00,start
44,building 1,computer,2013-03-14 20:20:02+00:00,start
25,building 1,broadband router,2017-04-01 09:07:11+01:00,end
15,building 1,kettle,2017-04-26 18:32:46+01:00,end


### Training Config

In [14]:
from nilmtk.api import API
from nilmtk.disaggregate import Mean
from nilmtk_contrib.disaggregate import DAE,Seq2Point, Seq2Seq, RNN

epochs = 1
batch_size = 32
apps = ['microwave', 'dish washer', 'kettle', 'broadband router',
        'computer monitor','laptop computer','external hard disk','computer']

config = {
    'power': {
        'mains': ['apparent','active'],
        'appliance': ['apparent','active']
    },
    'sample_rate': 10,

    'appliances': apps,
    'methods': {
        'RNN':RNN({'n_epochs':epochs,'batch_size':batch_size}),
        'DAE':DAE({'n_epochs':epochs,'batch_size':batch_size}),
        'Seq2Point':Seq2Point({'n_epochs':epochs,'batch_size':batch_size}),
        'Seq2Seq':Seq2Seq({'n_epochs':epochs,'batch_size':batch_size}),
        'Mean': Mean({}),
    },
    'train': {
        'datasets': {
            'Dataport': {
                'path': 'ukdale2.h5',
                'buildings': {
                    1: {
                        'start_time': '2016-11-01',
                        'end_time': '2017-02-01'
                    },
                    2: {
                        'start_time': '2013-05-20',
                        'end_time': '2013-08-20'
                        },
                    5: {
                        'start_time': '2014-07-01',
                        'end_time': '2014-10-01'
                        },
                }
            }
        }
    },
    'test': {
        'datasets': {
            'Datport': {
                'path': 'ukdale2.h5',
                'buildings': {
                    1: {
                        'start_time': '2017-02-15',
                        'end_time': '2017-03-08'
                    },
                    2: {
                        'start_time': '2013-09-15',
                        'end_time': '2013-10-03'
                    },
                    5: {
                        'start_time': '2014-10-15',
                        'end_time': '2014-11-03'
                        },
                }
            }
        },
        'metrics':['mae', 'rmse']
    }
}

df = next(ukdale.buildings.elec['microwave'].load())

,0,1,2,3
8,building 1,dish washer,2012-11-09 22:28:18+00:00,start
14,building 1,kettle,2012-11-09 22:28:18+00:00,start
34,building 1,laptop computer,2012-11-16 11:29:01+00:00,start
24,building 1,broadband router,2012-12-14 22:21:32+00:00,start
0,building 1,microwave,2012-12-14 22:21:33+00:00,start
28,building 1,computer monitor,2012-12-14 22:21:33+00:00,start
40,building 1,external hard disk,2013-03-14 20:20:02+00:00,start
44,building 1,computer,2013-03-14 20:20:02+00:00,start
25,building 1,broadband router,2017-04-01 09:07:11+01:00,end
15,building 1,kettle,2017-04-26 18:32:46+01:00,end
